<a href="https://colab.research.google.com/github/ProjectProRepo/How-to-Build-an-FAQ-ChatBot-using-Open-source-LLM-/blob/main/QuizGen_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install gradio transformers accelerate pypdf PyPDF2 bitsandbytes

In [1]:
import gradio as gr
from PyPDF2 import PdfReader
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

In [ ]:
from huggingface_hub import notebook_login
notebook_login()  # Login with your HF token

In [5]:
# Load LLM
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    load_in_8bit=True
)

llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Extract chunks from PDF
def extract_text_chunks(pdf_file, chunk_size=1500, overlap=200):
    reader = PdfReader(pdf_file)
    full_text = ""
    for page in reader.pages:
        full_text += page.extract_text() or ""

    chunks = []
    start = 0
    while start < len(full_text):
        end = start + chunk_size
        chunks.append(full_text[start:end])
        start += chunk_size - overlap
    return chunks

In [ ]:
# Find relevant chunk using simple keyword match
def find_relevant_chunk(chunks, topic):
    best_score = 0
    best_chunk = ""
    topic_words = set(topic.lower().split())

    for chunk in chunks:
        chunk_words = set(chunk.lower().split())
        score = len(topic_words.intersection(chunk_words))
        if score > best_score:
            best_score = score
            best_chunk = chunk
    return best_chunk


In [ ]:
# Generate quiz
def generate_quiz_from_pdf(pdf_file, topic):
    chunks = extract_text_chunks(pdf_file)
    relevant_chunk = find_relevant_chunk(chunks, topic)

    prompt = (
        f"You're an AI tutor. Based on the following content:\n\n"
        f"{relevant_chunk}\n\n"
        f"Create a quiz on the topic '{topic}' with 3 multiple-choice questions. "
        f"Each question must have 4 options and indicate the correct answer."
    )

    result = llm(prompt)[0]["generated_text"]
    return result.replace(prompt, "").strip()

In [3]:
# Gradio UI
iface = gr.Interface(
    fn=generate_quiz_from_pdf,
    inputs=[
        gr.File(file_types=[".pdf"], label="Upload a PDF (up to 22MB)"),
        gr.Textbox(lines=1, placeholder="Enter topic (e.g., Recursion, Sorting, etc.)", label="Quiz Topic")
    ],
    outputs="text",
    title="🧠 PDF-to-Quiz Generator",
    description="Upload a large PDF (like a textbook or research paper) and get a quiz generated from a topic of your choice."
)

if __name__ == "__main__":
    iface.launch()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://628cbce96d709918f8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
